#### Installing PySpark and findSpark

In [1]:
! pip install pyspark

In [2]:
# mandatory step so that jupyter notebook finds your pyspark
! pip install findspark pyspark

In [3]:
import os
os.chdir('F:\CDAC\Big Data\pyspark\Titanic')

In [4]:
import pyspark

# mandatory step
import findspark
findspark.init()

In [5]:
# Importing SparkSession - entry point for spark

from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('Understanding Spark Session').master('local[4]').getOrCreate()

In [7]:
spark

In [8]:
sc = spark.sparkContext

# DATAFRAME

### Creating DataFrame from RDD

In [9]:
iphones_rdd = sc.parallelize([
                                ("12", 2020, 2, 50000),
                                ("13", 2021, 2, 60000),
                                ("14 pro", 2022, 3, 100000),
                                ("15 pro max", 2023, 3, 150000)
                            ])

In [10]:
col_names = ['Model', 'Year of Manufacturing', 'No. of Camera', 'Price']

In [11]:
iphones_df = spark.createDataFrame(iphones_rdd, schema = col_names)
type(iphones_df)

pyspark.sql.dataframe.DataFrame

### Creating DataFrame from a file(csv, json, txt, etc...)

In [12]:
# .read have many more functionalities for other types of files.
# for more refer documentation

# inferSchema - tries to automatically determine column of one type (i.e. if all nunm entries in col1 then make it numeric)

titanic = spark.read.csv("titanic_train.csv", header = True, inferSchema = True)
titanic

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

In [13]:
titanic.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|          6|       0|     3|    Moran, Mr. James|  male|NULL|    0|    0|      

## TRANFORMATIONS AND ACTIONS ON DATAFRAME

### SELECT - Transformation

In [14]:
# select any one column from the DataFrame

passengers = titanic.select('Name')

### SHOW - Action

In [15]:
# displays the first 20 rows of the given DataFrame

passengers.show()

+--------------------+
|                Name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
|    Moran, Mr. James|
|McCarthy, Mr. Tim...|
|Palsson, Master. ...|
|Johnson, Mrs. Osc...|
|Nasser, Mrs. Nich...|
|Sandstrom, Miss. ...|
|Bonnell, Miss. El...|
|Saundercock, Mr. ...|
|Andersson, Mr. An...|
|Vestrom, Miss. Hu...|
|Hewlett, Mrs. (Ma...|
|Rice, Master. Eugene|
|Williams, Mr. Cha...|
|Vander Planke, Mr...|
|Masselmani, Mrs. ...|
+--------------------+
only showing top 20 rows



In [16]:
# show limited rows

passengers.show(3)

+--------------------+
|                Name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
+--------------------+
only showing top 3 rows



### FILTER - Transformation

In [17]:
# filters the dataframe on give condition for which the condition returns TRUE

survived_people = titanic.filter(titanic.Survived == 1)

In [18]:
survived_people.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+--------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|    Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+--------+-----+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599| 71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|   7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|    53.1| C123|       S|
|          9|       1|     3|Johnson, Mrs. Osc...|female|27.0|    0|    2|          347742| 11.1333| NULL|       S|
|         10|       1|     2|Nasser, Mrs. Nich...|female|14.0|    1|    0|          237736| 30.0708| NULL|       C|
|         11|       1|     3|Sandstrom, Miss. ...|female| 4.0|    1|    

### GROUPBY - Transformation

In [19]:
# groups data on the given condition

survived_nonsurvived = titanic.groupby('Survived')

In [20]:
# returns GroupedData object and not DataFrame Object 

type(survived_nonsurvived)

pyspark.sql.group.GroupedData

### COUNT - Transformation

In [21]:
# returns the count of the values in the DataFrame

survived_nonsurvived.count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



### ORDER BY - Transformation

In [22]:
# sorts the df on alphabetically on the basis of given condition/column

sorted_names = titanic.orderBy('Name')

In [23]:
sorted_names.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|  Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|        147|       1|     3|"Andersson, Mr. A...|  male|27.0|    0|    0|350043|7.7958| NULL|       S|
|        519|       1|     2|"Angle, Mrs. Will...|female|36.0|    1|    0|226875|  26.0| NULL|       S|
|        291|       1|     1|"Barber, Miss. El...|female|26.0|    0|    0| 19877| 78.85| NULL|       S|
|        625|       0|     3|"Bowen, Mr. David...|  male|21.0|    0|    0| 54636|  16.1| NULL|       S|
|        508|       1|     1|"Bradley, Mr. Geo...|  male|NULL|    0|    0|111427| 26.55| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
only showing top 5 rows



### DROPDUPLICATES - Transformation

In [24]:
# removes duplicate records from the DataFrame

new_titanic = titanic.select('Name','Age','Survived').dropDuplicates()

In [25]:
new_titanic.show(3)

+--------------------+----+--------+
|                Name| Age|Survived|
+--------------------+----+--------+
|  Giglio, Mr. Victor|24.0|       0|
|Goodwin, Master. ...| 1.0|       0|
|Goldsmith, Mr. Fr...|33.0|       0|
+--------------------+----+--------+
only showing top 3 rows



### withColumnRenamed - Transformation

In [26]:
# used to rename the column of the DataFrame 

new_titanic = new_titanic.withColumnRenamed('Name', 'Passenger Name')

In [27]:
new_titanic.show(3)

+--------------------+----+--------+
|      Passenger Name| Age|Survived|
+--------------------+----+--------+
|  Giglio, Mr. Victor|24.0|       0|
|Goodwin, Master. ...| 1.0|       0|
|Goldsmith, Mr. Fr...|33.0|       0|
+--------------------+----+--------+
only showing top 3 rows



### PRINTSCHEMA - Action

In [28]:
# returns the types of columns in DataFrame

titanic.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



### COLUMNS - Action

In [29]:
# returns names of all columns in the DataFrame

titanic.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

### DESCRIBE - Transformation

In [41]:
# show numerical statistics of DataFrame

titanic.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                NULL|  NULL| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

## USING DATAFRAME TO RUN SQL QUERIES

### CREATE OR REPLACE TEMP VIEW

In [66]:
# creates a sql table from the dataframe

titanic.createOrReplaceTempView("titanic_table")

In [69]:
# running sql query using spark.sql 
# it is a transformation 

titanic_sql = spark.sql("Select Name, Sex, Age from titanic_table where Survived = 1 ")

### Output by SHOW(DataFraame Action) and TAKE(RDD Action)
show - Dataframe
take - rdd array of rows

In [73]:
titanic_sql.show(3)

+--------------------+------+----+
|                Name|   Sex| Age|
+--------------------+------+----+
|Cumings, Mrs. Joh...|female|38.0|
|Heikkinen, Miss. ...|female|26.0|
|Futrelle, Mrs. Ja...|female|35.0|
+--------------------+------+----+
only showing top 3 rows



In [74]:
titanic_sql.take(3)

[Row(Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age=38.0),
 Row(Name='Heikkinen, Miss. Laina', Sex='female', Age=26.0),
 Row(Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age=35.0)]

#### Another way to run same query

In [53]:
query = '''Select Name, Sex, Age from titanic_table where Survived = 1'''

In [65]:
titanic_sql = spark.sql(query)
titanic_sql.show(3)

+--------------------+----+
|                Name| Age|
+--------------------+----+
|McCarthy, Mr. Tim...|54.0|
|Wheadon, Mr. Edwa...|66.0|
|Ostby, Mr. Engelh...|65.0|
+--------------------+----+
only showing top 3 rows



### Another Query

In [59]:
query = '''Select Age, count(Name) as Passengers from titanic_table group by Age'''

In [60]:
titanic_sql = spark.sql(query)
titanic_sql.show(5)

+----+----------+
| Age|Passengers|
+----+----------+
| 8.0|         4|
|70.0|         2|
| 7.0|         3|
|20.5|         1|
|49.0|         6|
+----+----------+
only showing top 5 rows



### Another Query

In [61]:
query = '''Select Name, Age from titanic_table where Age > 50 and Sex = "male" '''

In [64]:
titanic_sql = spark.sql(query)
titanic_sql.collect()

[Row(Name='McCarthy, Mr. Timothy J', Age=54.0),
 Row(Name='Wheadon, Mr. Edward H', Age=66.0),
 Row(Name='Ostby, Mr. Engelhart Cornelius', Age=65.0),
 Row(Name='Coxon, Mr. Daniel', Age=59.0),
 Row(Name='Goldschmidt, Mr. George B', Age=71.0),
 Row(Name='Connors, Mr. Patrick', Age=70.5),
 Row(Name='White, Mr. Percival Wayland', Age=54.0),
 Row(Name='Bateman, Rev. Robert James', Age=51.0),
 Row(Name='Meo, Mr. Alfonzo', Age=55.5),
 Row(Name='Williams, Mr. Charles Duane', Age=51.0),
 Row(Name='Van der hoef, Mr. Wyckoff', Age=61.0),
 Row(Name='Smith, Mr. James Clinch', Age=56.0),
 Row(Name='Green, Mr. George Henry', Age=51.0),
 Row(Name='Sjostedt, Mr. Ernst Adolf', Age=59.0),
 Row(Name='Carter, Rev. Ernest Courtenay', Age=54.0),
 Row(Name='Stead, Mr. William Thomas', Age=62.0),
 Row(Name='Taussig, Mr. Emil', Age=52.0),
 Row(Name='Duane, Mr. Frank', Age=65.0),
 Row(Name='Moraweck, Dr. Ernest', Age=54.0),
 Row(Name='Nysveen, Mr. Johan Hansen', Age=61.0),
 Row(Name='Widegren, Mr. Carl/Charles Pe